In [39]:
import pandas as pd
import os
import json
import numpy as np

In [40]:
with open("../siqa_task.json", mode="r") as f:
    siqa_task = json.load(f)

siqa_data = siqa_task['examples']

In [47]:
# labels_df = pd.DataFrame(columns=['idx', 'sentence', 'label'])
idx = []
sentences = []
targets = []
for i, item in enumerate(siqa_data):
    s = item['input']
    # print(list(item['target_scores'].values()))
    label = np.argmax(list(item['target_scores'].values()))
    idx.append(i)
    sentences.append(s)
    targets.append(label)

labels_df = pd.DataFrame({'idx': idx, 'sentences': sentences, "targets": targets})
labels_df

,idx,sentences,targets
0,0,Tracy didn't go home that evening and resisted...,2
1,1,Sydney walked past a homeless woman asking for...,0
2,2,Sasha protected the patients' rights by making...,1
3,3,Jordan was in charge of taking the food on the...,0
4,4,Kendall opened their mouth to speak and what c...,2
...,...,...,...
1949,1949,Bailey destroyed the enemy's bunker and won th...,0
1950,1950,Kai told Kendall the reason about the accident...,1
1951,1951,Kai improved Jan's picture and she loved how i...,2
1952,1952,Tracy's kids wanted ice cream so Aubrey fed th...,1


In [8]:
base_csv_pth = "../baselines"
male_csv = "0-shots-Male.csv"
female_csv = "0-shots-Female.csv"
amb_csv = "0-shots-Ambiguous.csv"
baseline_csv = "0-shots-Baseline.csv"

In [27]:
baseline_df = pd.read_csv(os.path.join(base_csv_pth, baseline_csv)).rename(columns={'Unnamed: 0': 'idx'})
male_df = pd.read_csv(os.path.join(base_csv_pth, male_csv)).rename(columns={'Unnamed: 0': 'idx'})
female_df = pd.read_csv(os.path.join(base_csv_pth, female_csv)).rename(columns={'Unnamed: 0': 'idx'})
amb_df = pd.read_csv(os.path.join(base_csv_pth, amb_csv)).rename(columns={'Unnamed: 0': 'idx'})

In [28]:
baseline_df

,idx,sentences,choices,scores
0,0,\nQ: Tracy didn't go home that evening and res...,"['Make a new plan', 'Go home and see Riley', '...",0
1,1,\nQ: Sydney walked past a homeless woman askin...,"['Sympathetic', 'Like a person who was unable ...",0
2,2,\nQ: Sasha protected the patients' rights by m...,"['Write new laws', 'Get petitions signed', 'Li...",1
3,3,\nQ: Jordan was in charge of taking the food o...,['Horrible that he let his friends down on the...,0
4,4,\nQ: Kendall opened their mouth to speak and w...,"['A very quiet person', 'A very passive person...",0
...,...,...,...,...
1949,1949,\nQ: Bailey destroyed the enemy's bunker and w...,"['Spy on enemy troop movements', 'Give the ene...",2
1950,1950,\nQ: Kai told Kendall the reason about the acc...,"['More at ease', 'Like they got something off ...",0
1951,1951,\nQ: Kai improved Jan's picture and she loved ...,"['Wanted to be helpful', 'Wanted to Jan to be ...",2
1952,1952,\nQ: Tracy's kids wanted ice cream so Aubrey f...,"['Tell her kids to say thank you', 'Get ice cr...",1


In [52]:
# misclassifications male
merged_df = pd.merge(baseline_df, male_df, on=['idx'], suffixes=('_df1', '_df2'))
merged_df['sentences_df1'] = merged_df['sentences_df1'].apply(lambda x: x.split("\n")[1].split(":")[1])
merged_df['sentences_df2'] = merged_df['sentences_df2'].apply(lambda x: x.split("\n")[1].split(":")[1])

merged_df = pd.merge(merged_df, labels_df[["idx", "targets"]], on=['idx'])

# print(merged_df.head())
# Select rows where scores in the two dataframes are not equal
unequal_rows = merged_df[merged_df['scores_df1'] != merged_df['scores_df2']]

unequal_rows.head()
# Display the result
unequal_rows.to_csv(os.path.join(base_csv_pth, "misclassifications", "base-male.csv"), index=False)

In [54]:
# misclassifications male
merged_df = pd.merge(baseline_df, female_df, on=['idx'], suffixes=('_df1', '_df2'))
merged_df['sentences_df1'] = merged_df['sentences_df1'].apply(lambda x: x.split("\n")[1].split(":")[1])
merged_df['sentences_df2'] = merged_df['sentences_df2'].apply(lambda x: x.split("\n")[1].split(":")[1])

merged_df = pd.merge(merged_df, labels_df[["idx", "targets"]], on=['idx'])
# print(merged_df.head())
# Select rows where scores in the two dataframes are not equal
unequal_rows = merged_df[merged_df['scores_df1'] != merged_df['scores_df2']]

unequal_rows.to_csv(os.path.join(base_csv_pth, "misclassifications", "base-female.csv"), index=False)
# unequal_rows.head()

In [33]:
# misclassifications male
merged_df = pd.merge(baseline_df, amb_df, on=['idx'], suffixes=('_df1', '_df2'))
merged_df['sentences_df1'] = merged_df['sentences_df1'].apply(lambda x: x.split("\n")[1].split(":")[1])
merged_df['sentences_df2'] = merged_df['sentences_df2'].apply(lambda x: x.split("\n")[1].split(":")[1])

merged_df = pd.merge(merged_df, labels_df[["idx", "targets"]], on=['idx'])
# print(merged_df.head())
# Select rows where scores in the two dataframes are not equal
unequal_rows = merged_df[merged_df['scores_df1'] != merged_df['scores_df2']]

unequal_rows.to_csv(os.path.join(base_csv_pth, "misclassifications", "base-amb.csv"), index=False)